# Si on utilise cette fonction sur un fichier obtenu automatiquement, utiliser les fonctions suivantes.

In [3]:
def xe_a_changer(csv_lu):
    
    """
    Fonction récupérant le contenu d'un CSV avec les colones suivantes :
        - "ID"
        - "TOKEN"
        - "LEMMES"
        - "POS"
        
    Cette fonction boucle sur chaque token et chaque fois garde un trio de tokens
    dont les numéros se suivent (en excluant la ponctuation). Puis, sur ce trio, on
    teste des conditions : si le premier et le troisième == "Xe" mais pas le deuxième,
    alors le deuxième token doit être considéré comme Xe. S'il n'a pas de lemme ("INC"),
    alors son lemme devient exactement le mot-forme (en majuscules).
    
    La fonction ajoute alors la nouvelle forle dictionnaire du token comme valeur.
    Une fois tous les tokens analysés, la fonction retourne le dictionnaire contenant
    tous les tokens modifiés.
    
    :param csv_lu: Le contenu du fichier CSV sur lequel on souhaite
            effectuer des vérifications, extrait et lu par la fonction principale.
            
    :return: Retourne un dictionnaire dont les clés sont les numéros des tokens
            contenus et les valeurs la ligne entière du CSV modifiée par la fonction.
    
    """
    
    dico_mots = {}
    dico_local = {}
    dico_modifies = {}
    id_dico_local = 1
    compteur = 0
    total = 0
    
    # On boucle sur les lignes du CSV d'entrée pour créer un dictionnaire général
    # (dico_mots) à exploiter et on ajoute 1 au compte total de tokens.
    for row in csv_lu:
        total += 1
            
        # Condition : on ne mettra le token dans le dictionnaire général que si ce n'est
        # pas un signe de ponctuation.
        if row['POS'] != "Fs" and row['POS'] != "Fo" and row['POS'] != "Fw":
                
            # On transfère les clés et valeurs dans un dictionnaire local,
            # de manière à avoir un dictionnaire général avec
            # clé = numéro d'ordre des tokens (pas identifiant des tokens)
            # valeurs = dictionnaire identique à la ligne du CSV d'origine.
                
            dico_local['ID'] = row['ID']
            dico_local['TOKEN'] = row['TOKEN']
            dico_local['LEMMES'] = row['LEMMES']
            dico_local['POS'] = row['POS']
            identifiant = row['ID']

                # On ajoute l'unité au dictionnaire général,
                # puis on remet le dictionnaire local à vide,
                # (j'ai eu de grosses erreurs avant d'y penser).
            dico_mots[id_dico_local] = dico_local
            dico_local = {}
            
            id_dico_local += 1
                
    # On boucle sur les clés du dictionnaire de tokens.
    for identifiant in dico_mots.keys():
        # S'il n'y a pas deux tokens après lui, on ne fait rien.
        if identifiant + 2 in dico_mots.keys():
            
            # On stocke les trois tokens dans trois variables.
            mot_precedent = dico_mots[identifiant]
            mot_cible = dico_mots[identifiant + 1]
            mot_suivant = dico_mots[identifiant + 2]
            
            # Si le token n'est pas exactement "Xe" mais est encadré par deux
            # tokens étant strictement "Xe", on le modifie.
            if mot_precedent['POS'] == "Xe" and mot_cible['POS'] != "Xe" and mot_suivant['POS'] == "Xe":
                
                # Si le mot-cible a plusieurs analyses de POS, l'une d'elle étant "Xe",
                # et plusieurs lemmes possibles, on regarde quelle est la position du
                # POS "Xe" et on prend le lemme qui lui correspond.
                if "Xe" in mot_cible['POS'] and "|" in mot_cible["LEMMES"]:
                    index_pos = mot_cible['POS'].split("|").index("Xe")
                    liste_lemmes = mot_cible["LEMMES"].split("|")
                    mot_cible["LEMMES"] = liste_lemmes[index_pos]
                    
                # Sinon, si Analog n'a pas trouvé de lemme, on prend le mot-forme
                # et on le passe en majuscules pour faire un lemme.
                elif mot_cible['LEMMES'] == "INC":
                    mot_cible['LEMMES'] = mot_cible['TOKEN'].upper()
                
                # Dans les deux cas, le POS du mot-cible devient "Xe" strictement.
                mot_cible['POS'] = "Xe"
                
                # On ajoute le mot-cible au dictionnaire des tokens modifiés
                # et on ajoute 1 au compteur de tokens modifiés.
                dico_modifies[mot_cible['ID']] = mot_cible
                compteur += 1
    
    # On calcule et on donne le nombre et le pourcentage de tokens modifiés par la fonction.
    pourcent_traites = compteur * 100 / total
    print(compteur, ":", round(pourcent_traites, 1), "%")
    
    # On retourne le dictionnaire des tokens modifiés.
    return dico_modifies

In [1]:
def desambiguiser(csv_entree, csv_sortie):
    
    """
        Fonction récupérant un CSV dont chaque ligne est un token lemmatisé,
        avec les colonnes telles quelles :
        
            - "ID" --- étant le numéro constituant la valeur du @n
                    de l'élément <w> dans le XML dont les tokens ont été extraits
                    (en principe, chaque token a un numéro unique).
                    
            - "TOKEN" --- étant le texte de l'élément <w> original, avec
                    abréviations résolues et graphie modernisée s'il y a lieu.
                    
            - "LEMMES" --- étant le lemme préconisé par Analog, ou bien les lemmes
                    entre lesquels Analog hésite, séparés par des '|'.
                    
            - "POS" --- étant la valeur POS préconisée par Analog, ou bien les
                    différentes valeurs entre lesquelles Analog hésite, séparées par
                    des '|'.
        
        La fonction crée à partir du CSV d'entrée un dictionnaire qu'elle transmet à la
        fonction secondaire chargée d'analyser les tokens et les modifier si besoin et
        récupère un dictionnaire contenant uniquement les tokens modifiés.
        
        Puis elle produit un autre fichier CSV. Si le token se trouve dans le dictionnaire
        retourné par la fonction secondaire, c'est cette version qui sera écrite dans le
        nouveau CSV. Sinon, la version originale sera copiée.
        
        :param csv_entree: Le chemin interne du fichier CSV sur lequel on souhaite
            effectuer des vérifications.
            
        :param csv_sortie: Le chemin interne du fichier CSV dans lequel écrire
            les tokens modifiés.
            
    """
        
    import csv
    
    # Colonnes du futur CSV de sortie.
    colonnes = [
            'ID',
            'TOKEN',
            'Modifié',
            'LEMMES',
            'POS'
        ]
    # Dictionnaire local pour stocker les nouvelles formes des tokens modifiés.
    row_modifie = {}
    
    # Ouvrir le CSV d'entrée et le lire.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire, delimiter=";")
        
        # Appeler la fonction secondaire et récupérer le dictionnaire des
        # tokens modifiés par elle.
        dico_modifies = xe_a_changer(csv_lu)
    
    # Ouvrir à nouveau le CSV d'entrée.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire, delimiter=";")

        # Ouvrir le fichier de sortie et le lire en mode écriture.
        with open(csv_sortie, 'w') as csv_a_ecrire:
            a_ecrire = csv.DictWriter(csv_a_ecrire, fieldnames = colonnes)
            # Écrire les noms des colonnes.
            a_ecrire.writeheader()
            
            # Boucler sur les lignes du CSV d'entrée.
            for row in csv_lu:
                
                # Si le numéro du token-cible se trouve parmi
                # les tokens modifiés, continuer :
                if row['ID'] in dico_modifies.keys():
                    # Stocker la forme modifiée dans une variable.
                    row_modifie = dico_modifies[row['ID']]
                    
                    # Écrire les nouvelles informations dans le CSV de sortie.
                    a_ecrire.writerow({
                        'ID' : row_modifie['ID'],
                        'TOKEN' : row_modifie['TOKEN'],
                        'Modifié' : "yes",
                        'LEMMES' : row_modifie['LEMMES'],
                        'POS' : row_modifie['POS']
                    })
                    # Remettre à vide le dictionnaire local.
                    row_modifie = {}
                    
                # Si le numéro du token-cible ne se trouve pas parmi les tokens modifiés,
                # écrire la version originelle du token dans le CSV de sortie.
                else:
                    a_ecrire.writerow({
                        'ID' : row['ID'],
                        'TOKEN' : row['TOKEN'],
                        'Modifié' : "no",
                        'LEMMES' : row['LEMMES'],
                        'POS' : row['POS']
                    })

In [4]:
desambiguiser(
    '/home/erminea/Documents/CONDE/GC/2021-05-27_GC_tokens.csv',
    '/home/erminea/Documents/CONDE/GC/2021-05-27_GC_tokens_xe_fait.csv'
)
# Vérifier les délimiteurs la prochaine fois, ils ne sont pas uniformes...

0 : 0.0 %


In [4]:


desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Basnage-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Basnage-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Berault-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Berault-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/instructions-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/instructions-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Merville-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Merville-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Pesnelle-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Pesnelle-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Rouille-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Rouille-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/ruines-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/ruines-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/tac-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/tac-xe-fait.csv'
)

desambiguiser(
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Terrien-base-orig.csv',
    '/home/erminea/Documents/CONDE/Encodage/2020-04-22-work/Terrien-xe-fait.csv'
)

2693 : 0.2 %
1116 : 0.2 %
0 : 0.0 %
31 : 0.0 %
355 : 0.0 %
2 : 0.0 %
0 : 0.0 %
0 : 0.0 %
173 : 0.0 %


# Si on fait des modifications sur un CSV dont les lemmes et pos ont été extraits d'un XML déjà resynchronisés, LES COLONNES SERONT DIFFÉRENTES : utiliser le fichier équivalent dans ../../../reprise_de_fichier_deja_lemmatise (même structure).